## Calculation of the VCI

In this example the Vegetation Condition Index (VCI) is calculated from the Enhanced Vegetation Index (EVI) gained from satellite remote sensing data. A MODIS vegetation product (MOD13Q1) with a temporal resolution of 16 days and a spatial resolution of 250m is used (https://lpdaac.usgs.gov/dataset_discovery/modis/modis_products_table/mod13q1). 

Hit `Shift + Enter` or the run button to execute the cells.

### Download and install Anaconda 3

To run this Jupyter Notebook you need python installed on your computer. We recommend to download and install Anaconda 3 (https://www.anaconda.com/download/) with the python 3.6 version as it includes a lot of usefule packages.

### Download MODIS data as Geotiffs from AppEEARS

Access the AppEEARS website and create a user account (free) for USGS in case you do not already have one. Click on __Extract__ and __Area Sample__ and start a new request.

![](http://www.un-spider.org/sites/default/files/snap1_adj.PNG)

1. Enter a request name. 
2. Define your region of interest by specifying a __ESRI shapefile__ (zip) or drawing a __polygon__. Country shapefiles can be downloaded at http://www.gadm.org/country.  
**Note:** The boundaries and names of the shapefiles do not imply official endorsement or acceptance by the United Nations.
3. Define the time period of your data: January __2000__ to __today__
4. Select the product: __MOD13Q1__ and the layers of interest: __EVI__ and __pixel_reliability__
5. Define the output as __Geotiff__ with __geographic Pojection__
6. Click on __Submit__

![](http://www.un-spider.org/sites/default/files/snap2_adj.PNG)

Click on __Explore__ to check the status of the request. When the status says "Done" you can click on the __Download__ symbol. __Select all__ ordered datasets and start downloading the data.

![](http://www.un-spider.org/sites/default/files/snap3_adj_new.PNG)

After the download store the files (.tif) in two folders: one folder for the __evi_data__ and one for the __pixel_reliability__.
It is reccomended to use an external hard drive to store your input and output data if possible. 
![](http://www.un-spider.org/sites/default/files/folders.PNG)

### Install packages

The modul __gdal__ needs to be installed separately and Bigtiffs must be supported. For that reason it is recommended to install gdal from __conda-forge__ (https://conda-forge.org/) as follows:
1. Open the __Anaconda Promt__ by searching for it in the windows start menu.
2. Type: `conda install -c conda-forge gdal` and hit `Enter`
![](http://www.un-spider.org/sites/default/files/gdal_inst.PNG)
3. Type `y` if you are asked if you want to proceed.

__Optional__: Create a virtual environment and install the package there if you plan to use python for different projects as some packages might interfere with each other. More information can be found __[here](https://github.com/yy/dviz-course/wiki/Create-and-manage-virtual-environments-with-Anaconda)__.

### Download this notebook and run it using Jupyter Notebook

Download the script (.ipynb) __[here](https://drive.google.com/drive/folders/1iQwiVaLQAuIm435Cli0DkdiubJqsl8X4?usp=sharing)__ and save it on your computer. Start Jupyter Notebook by searching for it in the Windows start menu or by typing `jupyter notebook` in the Anaconda Prompt and hitting `Enter`. After Jupyter Notebook opened in your web browser, search for the script on your computer and open it.

Every cell has to be executed individually. A cell, which is still processing is marked by a `*` and a finished cell with a number. Cells are executed by hitting `Shift + Enter` or by clicking on the run button.

### Load packages

In [1]:
from osgeo import gdal
import os, osr
import numpy as np
gdal.UseExceptions()

In [2]:
import matplotlib.pyplot as plt
from matplotlib import colors
%matplotlib inline

In [3]:
# The script creates some warnings due to the NA 
# in the data. They are ignored by executing this cell.
import warnings
warnings.filterwarnings('ignore')

### Set data paths 

Input data paths: ------------- ### TODO ### -------------

In [4]:
# Specify the name of your study area
studyarea = "Ghana"

In [5]:
# Specify the folder path where the EVI geotiffs
# are stored on your computer or external hard drive.
path_d = "D:/Ghana/evi_data/"

# Specify the folder path where the pixel reliability 
# geotiffs are stored on your computer or external hard drive.
path_c = "D:/Ghana/pixel_reliability"

Output data paths: ------------- ### TODO ### -------------

In [6]:
# Create and specify the folder where the output 
# png-files should be stored
path_png = "D:/Ghana/VCI_maps_Ghana_png"

# Create and specify the folder where the output
# tif-files should be stored
path_tif = "D:/Ghana/VCI_maps_Ghana_tif"

### Start processing

In the following cell, a function to write the data as geotiffs is defined. The geo information is extracted from the input data.

In [7]:
def array2raster(newRasterfn,array): 

    # Read columns from data array
    cols = array.shape[1]
    # Read rows from data array
    rows = array.shape[0]                                                         

    # Set the driver to Geotiff 
    driver = gdal.GetDriverByName('GTiff')                                        
    # Create raster with shape of array as float
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float64)         
    # Read geo information from input file
    outRaster.SetGeoTransform(expfile.GetGeoTransform())                          
    # Read band
    outband = outRaster.GetRasterBand(1)                                          
    # Set no data value to NA
    outband.SetNoDataValue(np.nan)                                                 
    outband.WriteArray(array) 
    # Set the projection according to the
    # input file projection
    outRaster.SetProjection(expfile.GetProjection())                             
    outband.FlushCache()                                                       

Within the following cell the Modis files are loaded per DOY (Day of the year), the cloudmask from the pixel reliability dataset  is applied, the EVI data is rescaled, the SVI is calculated and the SVI is written as smaller resolution PNG image and as Geotiff at the according output folders for every file.

__Note__: The script only works for files with file names according to the AppEEARS products, e.g. `MOD13Q1.006__250m_16_days_EVI_doy2001001_aid0001.tif` as the DOY information is extracted from the file name.

__Option__: If you want to use a different product (also geotiffs), you need to adapt the script where `file.split()` is used (three times). There, the files are split at the underscores and the 6th variable is used in this case to extract the DOY from the file name. If the DOY appears at a different position in your file name you need to adapt the number in the first squared brackets after the `file.split()`. Be aware that python uses zero-based indexing, so you need to start counting with zero.

In [14]:
# Create a list of numbers between 0 and 365
# Define the structure of the list as 001, 002, ... 
# equal to the DOY of the filename
for day in range(1, 365, 16):  
    day = str('%0.3d' % day)   
    
    
    # Reading an example file and 
    # define the shape of the data arrays
    
    filelist =[]
    
    # Create a list of files, which include the 
    # defined day in their filename for the EVI data
    for root, dirs, files in os.walk(path_d):  
         for file in files:                      
            if file.split("_")[6][7:] == day:
                filelist.append(file)
                
    # Read the first file of the file list as example file
    expfile = gdal.Open(os.path.join(path_d, filelist[0]))  
    
    # Create 3D numpy arrays according to the
    # example file dimensions
    evi = np.zeros((expfile.RasterYSize, expfile.RasterXSize, len(filelist)))         
    cloudmask = np.zeros((expfile.RasterYSize, expfile.RasterXSize, len(filelist)))   
    
    
    # Read the cloudmask for all years 
    
    rellist =[]
    # Creates a list of files, which include the 
    # defined day in their filename # for the pixel reliability data
    for root, dirs, files in os.walk(path_c):  
        for file in files:                       
            if file.split("_")[7][7:] == day:
                rellist.append(file)
    
    j = 0
    for file in rellist:
        
        # Open the file from the pixel reliability file list
        dataset_c = gdal.Open(os.path.join(path_c, file))    
        band_c = dataset_c.GetRasterBand(1)
        data_c = band_c.ReadAsArray()
        dataset_c = None
        
        # Write the data from each file to the array
        # dataset into the pre-defined 3D numpy array
        cloudmask[:,:,j] = data_c                         
                                                            
        j += 1
        
        # Delete intermdiate products
        del(data_c, band_c)                               
    
        # Exchange value 0 with value 1 (Pixel with value 0 and 1 are used)
        cloudmask[cloudmask == 0] = 1                     
        
        # Set value 2 to NA (Snow and Ice)
        cloudmask[cloudmask == 2] = np.nan                
        
        # Set value 3 to NA (Cloud)
        cloudmask[cloudmask == 3] = np.nan                 
        
        # Set no data value (= -1) to NA
        cloudmask[cloudmask == -1] = np.nan              
        
        # Set all values above 3 to NA
        cloudmask[cloudmask > 3] = np.nan                 
    
    
    # Reading the EVI data
    
    i = 0
    for file in filelist:
        
        # Open the file from the evi file list
        dataset = gdal.Open(os.path.join(path_d, file))      
        band = dataset.GetRasterBand(1)
        data = band.ReadAsArray()
        dataset = None
        
        # Apply the cloud mask
        data = data * cloudmask[:,:,i]                    
    
        # Rescale the data (to float)
        data = data * 0.0001                              
        
        # Set no data value to NA
        data[data == -0.3] = np.nan                       
        
        # Set negative values to NA
        data[data < 0] = np.nan                               
        
        # Write the data from each file to the array
        evi[:,:,i] = data                                 
        
        i += 1
        
        # Delete intermediate products
        del(data, band)                                   

    del(cloudmask)
    
    
    # Calculating the VCI
    
    # Calculate the mean of the 
    # array (NA are ignored)
    evimin = np.nanmin(evi, axis=2)                        
    
    # Calculate the standard deviation 
    # of the array (NA are ignored)
    evimax = np.nanmax(evi, axis=2)                      
    
    # Calculate the VCI
    vci = np.zeros((expfile.RasterYSize, expfile.RasterXSize, len(filelist)))   
    for k in range(len(filelist)):
        vci[:,:,k] = ((evi[:,:,k]-evimin)/(evimax-evimin))*100
    
    # Delete intermediate products
    del(evimin, evimax, evi)                              
    
    
    
    # Creating png images and geotiffs as outputs
    
    # Generate png images for every time step
    for l in range(len(filelist)):                        
        
        # Define the day and year of file for the output name of the image
        dayyear = day+filelist[l].split("_")[6][3:7]     
        year = filelist[l].split("_")[6][3:7]
        
        # Define the size of the figure (in inches)
        fig, ax = plt.subplots(figsize=(5,5))  
        plt.title("VCI (EVI) %s" %dayyear)     
        data = vci[:,:,l]
        
        # Define the colors
        cmap = colors.ListedColormap(['#8B0000', '#FF4500', '#FFFF00', '#9ACD32', '#008000'])  
        cax = ax.imshow(data, cmap=cmap)
        fig.colorbar(cax, cmap=cmap)
        
        # Save png images in
        # related folder
        plt.savefig(os.path.join(path_png, "VCI_"+studyarea+"_"+str(day)+"_"+str(year)+".png"), dpi=100)   
        plt.close()                                                                                        
        
        
        # Generating geotiffs as defined 
        # above for every time step
    
        # Save geotiffs related
        # folder
        newRasterfn = os.path.join(path_tif, "VCI_"+studyarea+"_"+str(day)+"_"+str(year)+".tif")  
        array = vci[:,:,l]                                                                        
        
        # Use pre-defined function to write geotiffs
        array2raster(newRasterfn,array)    
            
    del(vci)    # Delete VCI

***

__Note__: Using a large area of interest can lead to troubles with the memory (RAM) of your computer. You can use the Task Manager to keep an eye on the memory usage while running the script. If you encounter problems, consider using a smaller area of interest (shapefile or polygon in AppEEARS) or a satellite product with a smaller resolution (500m (MOD13A1), 1km (MOD13A2)).

This is an example of the output PNG images. The geotiffs can be read by QGIS, for example, and used to produce maps.
![](http://www.un-spider.org/sites/default/files/VCI_Ghana_001_2007.png)